In [1]:
from qwen_finetuning import QwenFineTuningConfig, QwenFineTuning

In [2]:
config = QwenFineTuningConfig(
    model_name="Qwen/Qwen3-8B",
    train_file="data/train.jsonl",
    output_dir="./model/v2",
    batch_size=4,                       # Memory-safe
    gradient_accumulation_steps=8,      # Effective batch = 32 (better for 90K)
    learning_rate=5e-5,                 # Slightly higher for larger batch
    warmup_ratio=0.05,                  # Less warmup needed with more steps
    lr_scheduler_type="cosine",
    num_epochs=1,                       # 1 epoch often sufficient for 90K
    max_length=512,
    lora_r=24,
    lora_alpha=48,
    lora_dropout=0.1,
    gradient_checkpointing=True,
)

In [3]:
# Create fine-tuning instance
finetuner = QwenFineTuning(config)

In [4]:
# Load data
train_data = finetuner.load_jsonl(config.train_file)
print(f"Training samples: {len(train_data)}")

Training samples: 86929


In [5]:
# Run training
finetuner.setup_model()

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

trainable params: 65,470,464 || all params: 8,256,205,824 || trainable%: 0.7930


In [6]:
finetuner.setup_trainer(train_data)

Formatting: 100%|██████████| 86929/86929 [00:10<00:00, 7903.66it/s]
/workspace/finetuning/myenv/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:453: UserWarning: Padding-free training is enabled, but the attention implementation is not set to 'flash_attention_2'. Padding-free training flattens batches into a single sequence, and 'flash_attention_2' is the only known attention mechanism that reliably supports this. Using other implementations may lead to unexpected behavior. To ensure compatibility, set `attn_implementation='flash_attention_2'` in the model configuration, or verify that your attention mechanism can handle flattened sequences.
  warnings.warn(
/workspace/finetuning/myenv/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:495: UserWarning: You are using packing, but the attention implementation is not set to 'flash_attention_2' or 'kernels-community/vllm-flash-attn3'. Packing flattens batches into a single sequence, and Flash Attention is the only known atten

Adding EOS to train dataset:   0%|          | 0/86929 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/86929 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/86929 [00:00<?, ? examples/s]

✓ Trainer ready (86929 samples, 2716 steps)


In [7]:
finetuner.train()


Starting training...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Step,Training Loss
50,1.551500
100,1.218500
150,1.197900
200,1.176500
250,1.162000
300,1.159400
350,1.142600
400,1.133300
450,1.130000
500,1.136400


✓ Training completed


In [8]:
finetuner.save_model()

✓ Model saved to ./model/v2


In [9]:
print(f"\n✅ Training complete! Model saved to: {config.output_dir}")


✅ Training complete! Model saved to: ./model/v2
